In [1]:
import numpy as np
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler


In [2]:
'''
pre-processes melanoma data

'''

'\npre-processes melanoma data\n\n'

In [3]:
# read in data

raw_data = pd.read_csv('Melanoma/total_PD_1.csv')
raw_data.head()

/Users/sophiesun/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,GSM2445716,GSM2445717,GSM2445718,GSM2445719,GSM2445720,GSM2445721,GSM2445722,GSM2445723,GSM2445724,...,SRR5088909,SRR5088911,SRR5088913,SRR5088914,SRR5088916,SRR5088920,SRR5088922,SRR5088924,SRR5088926,SRR5088929
0,5S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.039753,0.1036,0,0,0.0428182,0.138019,0.0139869,0.0539995,0.0914821,0.0727011
1,5_8S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,7SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,A1BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0298032,0.244701,0.172904,0.10584,0.188846,0.102993,0.0440417,0.151334,0.169748,0.0304227
4,A1BG-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410368,1.10834,1.55227,1.59396,1.20657,0.951995,0.117314,0.550993,1.18046,0.645802


In [4]:
# some re-formatting

# rename column

raw_data = raw_data.rename(columns={raw_data.columns[0]: 'patient'})

# transpose so columns are genes and rows are patients

t_data = raw_data.set_index('patient').T
print(t_data.head())

# split data

labels = t_data['Response']
clean_data = t_data.drop(columns = ['Response'])


patient    5S_rRNA 5_8S_rRNA  7SK A1BG A1BG-AS1 A1CF      A2M A2M-AS1 A2ML1  \
GSM2445716     NaN       NaN  NaN  NaN      NaN  NaN  12.4884     NaN   NaN   
GSM2445717     NaN       NaN  NaN  NaN      NaN  NaN  11.4637     NaN   NaN   
GSM2445718     NaN       NaN  NaN  NaN      NaN  NaN  12.1104     NaN   NaN   
GSM2445719     NaN       NaN  NaN  NaN      NaN  NaN  13.3247     NaN   NaN   
GSM2445720     NaN       NaN  NaN  NaN      NaN  NaN  14.4241     NaN   NaN   

patient    A2ML1-AS1  ... snoU2_19 snoU83B snoZ196 snoZ278 snoZ40 snoZ6  \
GSM2445716       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445717       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445718       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445719       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445720       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   

patient    snosnR66 uc_338 yR211F11.2 Response  
GSM2445716      NaN    Na

In [5]:
# drop any genes with > 90% missing data

clean_data = clean_data.dropna(thresh = clean_data.shape[0]*0.9, how='all', axis=1)
clean_data.head()

patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,12.4884,7.016,9.38074,6.82087,4.85041,2.64096,3.3414,11.309,7.60096,2.96289,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,11.4637,5.43465,8.65705,6.30912,4.61523,3.11273,4.2282,12.6538,9.57216,2.11273,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,12.1104,5.66996,9.85013,8.10566,6.16592,2.96952,3.55448,10.5177,5.84399,2.70649,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,13.3247,6.47553,8.68792,8.26729,5.91365,3.52134,3.52134,11.2229,7.77558,2.74373,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,14.4241,7.22529,9.96367,8.12778,7.74927,4.21583,4.55287,11.3331,8.26711,3.77526,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [6]:
# log2 transformation

log2_data = clean_data.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) and x > 0 else x)
log2_data.head()


patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,3.642517,2.810648,3.229701,2.769955,2.278107,1.401061,1.740452,3.499394,2.926182,1.567003,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,3.518996,2.442188,3.113875,2.657439,2.206401,1.638178,2.080045,3.661501,3.258844,1.079105,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,3.598169,2.503339,3.300142,3.018929,2.624316,1.570230,1.829640,3.394747,2.546954,1.436421,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,3.736027,2.694999,3.119011,3.047415,2.564050,1.816123,1.816123,3.488378,2.958950,1.456138,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,3.850413,2.853056,3.316678,3.022861,2.954060,2.075818,2.186776,3.502475,3.047384,1.916577,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [12]:
# missing data imputation

imputer = IterativeImputer()
imputed_data = imputer.fit_transform(log2_data)

imputed_data = pd.DataFrame(imputed_data, columns = log2_data.columns, index = log2_data.index)
imputed_data.head()

In [14]:
# z-score transformation (for PCA)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

scaled_data = pd.DataFrame(scaled_data).head()
scaled_data.head()


In [ ]:
# export

# imputed log2 transformed data
imputed_data.to_csv('imputed_data.csv')

# normalized version of above^ (for PCA)
scaled_data.to_csv('scaled_data.csv')

# true labels (response y/n)
labels.to_csv('labels.csv')